In [1]:
import ee
import geetools
import urllib.request
import pandas as pd
import numpy as np

In [2]:
ee.Authenticate()

Enter verification code: 4/1AbUR2VOUMfv-VR22m8kadX3XqicVIjnmyM-se1SQWmksGXv3MpftmHt12Aw

Successfully saved authorization token.


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions:

https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=X3HdwwlPOnxll13GWExVRdcvW1TK3qQsAuAeNjCqpqU&tc=pxt-Fa2tUQTGmupZziNzUSiAZfEbRhVojCjyKXr_4rY&cc=0nAGl69OyneFzAh-tdyudQyzQ9-agywwqXGUSnH9U_0

The authorization workflow will generate a code, which you should paste in the box below.

In [3]:
ee.Initialize()

## Export Copernicus over time


In [4]:
lat1 = 37.
lat2 = 41.
lon1 = -102.
lon2 = -109.
longitudes = []
latitudes = []
for i in np.arange(lon2,lon1,0.0427):
    for j in np.arange(lat1,lat2,0.0427):
        longitudes.append(i)
        latitudes.append(j)
        
locations = pd.DataFrame({
    'longitude': longitudes,
    'latitude': latitudes
})

In [5]:
locations.head(3)

,longitude,latitude
0,-109.0,37.0000
1,-109.0,37.0427
2,-109.0,37.0854


In [33]:
start_year = 2014
end_year = 2023

In [35]:
vis_min = 0  #Visualization settings for the thumbnail
vis_max = 10_000 #Visualization settings for the thumbnail
vis_bands = ['B4', 'B3', 'B2'] #Includes the bands for RGB
imageDimensions = '512x512' #Set thumbnail image size (can't be too big, or you run into problems)
start_location = 5000 # where you will start your loop based on the dataframe
end_location = 6000 # where you will end your loop based on the dataframe

for index,row in locations.iloc[start_location:end_location].iterrows():
    id_ = index
    longitude = row['longitude']
    latitude = row['latitude']
    center = ee.Geometry.Point(longitude,latitude)
    
    collection = []
    
    for year in range(start_year, end_year):
        # Import Sentinel dataset
        print(year)

        s2 = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
              .filterBounds(center)
              .sort('CLOUDY_PIXEL_PERCENTAGE',True)
              .filterDate(f'{year}-01-01', f'{year}-06-30')
              .first())
        collection.append(s2)
        
    for image in collection:

        # Create coordinates
        try:
            global sentinel_footprint
            sentinel_footprint = (ee.Geometry.Polygon(coords=[(longitude - 0.018,latitude - 0.018),
                                                          (longitude - 0.018,latitude + 0.018),
                                                         (longitude + 0.018,latitude - 0.018),
                                                         (longitude + 0.018,latitude + 0.018),]))
        except:
            print("No Footprint found")
            continue

        # Create a rectangular export area
        exportArea = (sentinel_footprint.centroid().buffer(2000).bounds())

        # Export Parameters
        visParams = {
            'region': exportArea,
            'crs': (image.select('B3').projection()),
            'dimensions': imageDimensions,
            'format': 'png',
            'min': vis_min,
            'max': vis_max,
            'bands': vis_bands,
            'gamma':1.5
        }

        # Export
        image_url = (image.getThumbURL(visParams))
        urllib.request.urlretrieve(image_url,f"..\\image{id_}_{round(longitude,2)}_{round(latitude,2)}_{year}.png")
        print(f'finished number {id_} at {round(longitude,2)} : {round(latitude,2)} : {year}')

2014
2015
2016
2017
2018
2019
2020
2021
2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
finished number 5000 at -106.74 : 37.77 : 2022
2014
2015
2016
2017
2018
2019
2020
2021
2022
finished number 5001 at -106.74 : 37.81 : 2022
finished number 5001 at -106.74 : 37.81 : 2022
finished number 5001 at -106.74 : 37.81 : 2022
finished number 5001 at -106.74 : 37.81 : 2022
finished number 5001 at -106.74 : 37.81 : 2022
finished number 5001 at -106.74 : 37.81 : 2022
finished number 5001 at -106.74 : 37.81 : 2022


KeyboardInterrupt: 

## Export Locations as CSV

In [ ]:
locations.to_csv(r"c:\downloads\centrepoints.csv")